**download SP500 stock data using yfinance**

In [1]:
from src.utils import sp500_symbols

symbols = sp500_symbols()
print(f'there are {len(symbols)} stocks in SP500 list.')

there are 503 stocks in SP500 list.


In [2]:
print(f'some SP500 stock symbol: {symbols[:5]}')

some SP500 stock symbol: ['MMM', 'AOS', 'ABT', 'ABBV', 'ACN']


In [ ]:
# for more info:  https://github.com/ranaroussi/yfinance/issues/2325

import time 

import yfinance as yf
from requests import Session
from requests_cache import CacheMixin, SQLiteCache
from requests_ratelimiter import LimiterMixin, MemoryQueueBucket
from pyrate_limiter import Duration, RequestRate, Limiter


class CachedLimiterSession(CacheMixin, LimiterMixin, Session):  #type:ignore
    pass

# https://help.yahooinc.com/dsp-api/docs/rate-limits
# Define multiple rate limits
yf_limiter = Limiter(
    RequestRate(60, Duration.MINUTE),      # Max 60 requests per minute
    RequestRate(360, Duration.HOUR),       # Max 360 requests per hour
    RequestRate(8000, Duration.DAY)         # Max 8000 requests per day
)

yf_session = CachedLimiterSession(
    limiter=yf_limiter,
    bucket_class=MemoryQueueBucket,
    backend=SQLiteCache("yfinance.cache"),
)

period = 'max'  # '5d'  '1y'  'max'

for symbol in symbols[:10]:
    tickers = [symbol]
    data = yf.download(
                        tickers=tickers,
                        period=period,
                        rounding = True,
                        group_by='ticker',
                        auto_adjust=True,
                        threads=1,  #type:ignore
                        session=yf_session
                    )
    
    time.sleep(7)

In [ ]:
import time


for ticker in symbols[:10]:
    try:
        data = yf.Ticker(ticker).history(period="1mo")
        time.sleep(5)  # Add delay
    except Exception as e:
        print(f"Rate limit hit. Waiting before retrying...")
        time.sleep(60)  # Wait longer before retrying


In [6]:
import yfinance as yf


period = 'max'  # '5d'  '1y'  'max'

stocks_df_raw = yf.download(tickers=symbols, group_by='Ticker', period=period, rounding=True)

[*********************100%***********************]  503 of 503 completed


In [11]:
assert stocks_df_raw is not None
stocks_df_raw.head()

Ticker     CTVA                        PPL                        ...  AIG  \
Price      Open High Low Close Volume Open High Low Close Volume  ... Open   
Date                                                              ...        
1962-01-02  NaN  NaN NaN   NaN    NaN  NaN  NaN NaN   NaN    NaN  ...  NaN   
1962-01-03  NaN  NaN NaN   NaN    NaN  NaN  NaN NaN   NaN    NaN  ...  NaN   
1962-01-04  NaN  NaN NaN   NaN    NaN  NaN  NaN NaN   NaN    NaN  ...  NaN   
1962-01-05  NaN  NaN NaN   NaN    NaN  NaN  NaN NaN   NaN    NaN  ...  NaN   
1962-01-08  NaN  NaN NaN   NaN    NaN  NaN  NaN NaN   NaN    NaN  ...  NaN   

Ticker                             GL                        
Price      High Low Close Volume Open High Low Close Volume  
Date                                                         
1962-01-02  NaN NaN   NaN    NaN  NaN  NaN NaN   NaN    NaN  
1962-01-03  NaN NaN   NaN    NaN  NaN  NaN NaN   NaN    NaN  
1962-01-04  NaN NaN   NaN    NaN  NaN  NaN NaN   NaN    NaN  
1962-01-05  NaN NaN   NaN    NaN  NaN  NaN NaN   NaN    NaN  
1962-01-08  NaN NaN   NaN    NaN  NaN  NaN NaN   NaN    NaN  

[5 rows x 2515 columns]

In [12]:
assert stocks_df_raw is not None
stocks_df_raw.tail()

Ticker       CTVA                                    PPL                       \
Price        Open   High    Low  Close     Volume   Open   High    Low  Close   
Date                                                                            
2025-05-05  62.34  62.98  61.91  62.41  2275900.0  36.20  36.38  35.84  36.28   
2025-05-06  62.34  62.84  62.00  62.45  2680800.0  36.24  36.41  36.07  36.12   
2025-05-07  62.67  62.82  62.02  62.48  3290700.0  36.05  36.58  36.05  36.33   
2025-05-08  64.63  67.25  63.82  66.86  6172800.0  36.18  36.23  35.54  35.62   
2025-05-09  66.71  67.93  66.57  67.79  4289100.0  35.73  35.74  35.21  35.51   

Ticker                 ...    AIG                                      GL  \
Price          Volume  ...   Open   High    Low  Close     Volume    Open   
Date                   ...                                                  
2025-05-05  3842500.0  ...  83.41  84.05  82.65  83.05  2619000.0  121.19   
2025-05-06  3957100.0  ...  82.51  83.31  82.32  82.53  2482300.0  120.74   
2025-05-07  4791600.0  ...  82.71  83.18  81.52  81.69  3325100.0  121.08   
2025-05-08  4920000.0  ...  82.22  83.00  81.70  81.82  3710000.0  121.90   
2025-05-09  4788100.0  ...  82.13  82.91  81.68  82.41  4926000.0  121.59   

Ticker                                        
Price         High     Low   Close    Volume  
Date                                          
2025-05-05  122.65  121.14  121.49  524400.0  
2025-05-06  122.24  119.84  120.34  657100.0  
2025-05-07  122.39  120.66  120.96  768400.0  
2025-05-08  122.87  120.60  121.23  604700.0  
2025-05-09  122.54  121.28  121.94  436100.0  

[5 rows x 2515 columns]

In [13]:
assert stocks_df_raw is not None
stocks_df_raw.columns

MultiIndex([('CTVA',   'Open'),
            ('CTVA',   'High'),
            ('CTVA',    'Low'),
            ('CTVA',  'Close'),
            ('CTVA', 'Volume'),
            ( 'PPL',   'Open'),
            ( 'PPL',   'High'),
            ( 'PPL',    'Low'),
            ( 'PPL',  'Close'),
            ( 'PPL', 'Volume'),
            ...
            ( 'AIG',   'Open'),
            ( 'AIG',   'High'),
            ( 'AIG',    'Low'),
            ( 'AIG',  'Close'),
            ( 'AIG', 'Volume'),
            (  'GL',   'Open'),
            (  'GL',   'High'),
            (  'GL',    'Low'),
            (  'GL',  'Close'),
            (  'GL', 'Volume')],
           names=['Ticker', 'Price'], length=2515)

**organize the multi-level column names**

In [14]:
assert stocks_df_raw is not None
stocks_df = stocks_df_raw.stack(level=0, future_stack=True).rename_axis(['Date', 'Ticker']).reset_index(level=1)

In [15]:
stocks_df.head()

Price,Ticker,Open,High,Low,Close,Volume
Date,,,,,,
1962-01-02,CTVA,NaN,NaN,NaN,NaN,NaN
1962-01-02,PPL,NaN,NaN,NaN,NaN,NaN
1962-01-02,JBL,NaN,NaN,NaN,NaN,NaN
1962-01-02,CARR,NaN,NaN,NaN,NaN,NaN
1962-01-02,GILD,NaN,NaN,NaN,NaN,NaN


In [16]:
print(f'there are {stocks_df.shape[0]:,} rows of stock records.')

there are 8,020,838 rows of stock records.


In [17]:
df = stocks_df.dropna()
print(f'there are {df.shape[0]:,} valid records.')

there are 4,308,245 valid records.


In [18]:
df.head()

Price,Ticker,Open,High,Low,Close,Volume
Date,,,,,,
1962-01-02,HON,0.00,1.05,1.05,1.05,40740.0
1962-01-02,MO,0.00,0.00,0.00,0.00,345600.0
1962-01-02,IP,0.00,0.89,0.87,0.87,51552.0
1962-01-02,DIS,0.06,0.06,0.06,0.06,841958.0
1962-01-02,XOM,0.00,0.09,0.09,0.09,902400.0


In [19]:
df.tail()

Price,Ticker,Open,High,Low,Close,Volume
Date,,,,,,
2025-05-09,LVS,39.78,40.12,39.62,39.79,5363700.0
2025-05-09,CEG,272.07,274.95,266.06,271.37,3244500.0
2025-05-09,SNPS,487.11,487.84,479.04,482.90,571500.0
2025-05-09,AIG,82.13,82.91,81.68,82.41,4926000.0
2025-05-09,GL,121.59,122.54,121.28,121.94,436100.0


In [20]:
df.describe()

Price,Open,High,Low,Close,Volume
count,4.308245e+06,4.308245e+06,4.308245e+06,4.308245e+06,4.308245e+06
mean,4.977447e+01,5.044368e+01,4.926039e+01,4.986493e+01,6.380172e+06
std,1.575594e+02,1.594162e+02,1.556959e+02,1.575684e+02,4.027378e+07
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,4.190000e+00,4.340000e+00,4.220000e+00,4.280000e+00,4.976000e+05
50%,1.731000e+01,1.756000e+01,1.708000e+01,1.733000e+01,1.479700e+06
75%,4.800000e+01,4.853000e+01,4.745000e+01,4.801000e+01,3.926703e+06
max,9.914170e+03,9.964770e+03,9.794000e+03,9.924400e+03,9.230856e+09


**clean the data**

In [21]:
df['Volume'] = df['Volume'].fillna(0).astype(int)

df.head()

/tmp/ipykernel_1356735/2435763008.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Volume'] = df['Volume'].fillna(0).astype(int)


Price,Ticker,Open,High,Low,Close,Volume
Date,,,,,,
1962-01-02,HON,0.00,1.05,1.05,1.05,40740
1962-01-02,MO,0.00,0.00,0.00,0.00,345600
1962-01-02,IP,0.00,0.89,0.87,0.87,51552
1962-01-02,DIS,0.06,0.06,0.06,0.06,841958
1962-01-02,XOM,0.00,0.09,0.09,0.09,902400


**set 'Date' as a column**

In [22]:
df.reset_index(inplace=True)
df.head()

Price,Date,Ticker,Open,High,Low,Close,Volume
0,1962-01-02,HON,0.00,1.05,1.05,1.05,40740
1,1962-01-02,MO,0.00,0.00,0.00,0.00,345600
2,1962-01-02,IP,0.00,0.89,0.87,0.87,51552
3,1962-01-02,DIS,0.06,0.06,0.06,0.06,841958
4,1962-01-02,XOM,0.00,0.09,0.09,0.09,902400


**check db table info**

In [23]:
from src.create_tables import StockPrice

column_info = {c.name: c.type for c in StockPrice.__table__.columns}
column_info

{'id': Integer(),
 'ticker': String(length=10),
 'trade_date': Date(),
 'open_price': Numeric(precision=10, scale=2),
 'high_price': Numeric(precision=10, scale=2),
 'low_price': Numeric(precision=10, scale=2),
 'close_price': Numeric(precision=10, scale=2),
 'volume': BigInteger()}

change dataframe column name to match db table column name in order to use to_sql

In [24]:
df_renamed = df.rename(columns={'Date': 'trade_date',
                                'Ticker': 'ticker',
                                'Open': 'open_price',
                                'High': 'high_price',
                                'Low': 'low_price',
                                'Close': 'close_price',
                                'Volume': 'volume'
                                })

In [25]:
df_renamed.head()

Price,trade_date,ticker,open_price,high_price,low_price,close_price,volume
0,1962-01-02,HON,0.00,1.05,1.05,1.05,40740
1,1962-01-02,MO,0.00,0.00,0.00,0.00,345600
2,1962-01-02,IP,0.00,0.89,0.87,0.87,51552
3,1962-01-02,DIS,0.06,0.06,0.06,0.06,841958
4,1962-01-02,XOM,0.00,0.09,0.09,0.09,902400


In [ ]:
from importlib import reload
from src import utils
reload(utils)

from src.utils import get_db_engine


engine = get_db_engine()

df_renamed.to_sql(name='stock_prices', con=engine, if_exists='append', index=False, chunksize=10_000)

-431

In [27]:
from sqlalchemy.orm import Session

session = Session(engine)

n_rows = session.query(StockPrice).count()

assert df_renamed.shape[0] == n_rows
print(f'There are {n_rows:,} rows in table {StockPrice.__tablename__}')

There are 4,308,245 rows in table stock_prices
